# Sheath Expansion Checks

Notebook made for testing two things related to teh sheath expansion theory Pete and I have come up with:
- How does the sheath expand around the probe tip laterally?
- What is the distribution of ions in the simulation?

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [2]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
skippable_scans = set() 
single_sims = set()

In [17]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/new_sheath_exp/*'
scans_searchstr = '*/*/sheath_exp_hg_fflwp/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 
                          'snumber', 'nproc', 'by', 'bz', 'rho01', 'rho02', 'rho'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)


skippable = {}
scans = list(scans)
scans.sort()

In [19]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-10.0
	[0,1]: angled_recessed_as/alpha_yz_-11.0
	[0,2]: angled_recessed_as/alpha_yz_-12.0
	[0,3]: angled_recessed_as/alpha_yz_-15.0
	[0,4]: angled_recessed_as/alpha_yz_-2.0
	[0,5]: angled_recessed_as/alpha_yz_-20.0
	[0,6]: angled_recessed_as/alpha_yz_-3.0
	[0,7]: angled_recessed_as/alpha_yz_-30.0
	[0,8]: angled_recessed_as/alpha_yz_-4.0
	[0,9]: angled_recessed_as/alpha_yz_-5.0
	[0,10]: angled_recessed_as/alpha_yz_-6.0
	[0,11]: angled_recessed_as/alpha_yz_-7.0
	[0,12]: angled_recessed_as/alpha_yz_-8.0
	[0,13]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as
	[1,0]: flat_flush_as/alpha_yz_-10.0
	[1,1]: flat_flush_as/alpha_yz_-12.0
	[1,2]: flat_flush_as/alpha_yz_-15.0
	[1,3]: flat_flush_as/alpha_yz_-2.0
	[1,4]: flat_flush_as/alpha_yz_-20.0
	[1,5]: flat_flush_as/alpha_yz_-3.0
	[1,6]: flat_flush_as/alpha_yz_-30.0
	[1,7]: flat_flush_as/alpha_yz_-4.0
	[1,8]: fl

---
## Run to Check 

In [86]:
spl_path = spice_dir / all_run_dirs[scans[1]][3]
print(spl_path)


/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0


In [87]:
'/'.join(str(spl_path.parent).split('/')[-3:])

'spice2/sheath_exp_hg_fflwp/flat_flush_as'

In [88]:
backups = list(spl_path.glob('backup*'))
n = len(backups)

for i, path in enumerate(backups):
    print(f'{i}, {i-n}: \t {"/".join(str(spl_path.parent).split("/")[-2:])}/{path.name}')

0, -24: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210123-1750
1, -23: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210124-1500
2, -22: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210125-1211
3, -21: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210126-0923
4, -20: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210127-0634
5, -19: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210128-0345
6, -18: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210129-0123
7, -17: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210129-2235
8, -16: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210130-1946
9, -15: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210131-1658
10, -14: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210201-1409
11, -13: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210203-0815
12, -12: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210204-0834
13, -11: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210205-1617
14, -10: 	 sheath_exp_hg_fflwp/flat_flush_as/backup_20210206-1329
15, -9: 	 sheath_exp

In [89]:
backup_path = list(spl_path.glob('backup*'))[-2]
backup_tfile = list(backup_path.glob('t-*.mat'))[0]
print(backup_path)
print(backup_tfile)

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0/backup_20210220-0212
/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0/backup_20210220-0212/t-alpha_yz_-2.0.mat


In [90]:
# tdata = td.Spice2TData(backup_tfile)
# tdata = None

In [97]:
backup_spl = spl.Splopter(backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                          store_dataframe_fl=True, check_voltage_error_fl=False)
backup_spl.parse_input()

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0/backup_20210220-0212) doesn't seem to be valid.
Continuing anyway.


In [92]:
fig, ax = plt.subplots()

ax.plot(backup_spl.tdata.t_dict['ProbePot'])

for i in range(4):
    line_pos = i * len(backup_spl.tdata.t_dict['ProbePot']) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)

In [93]:
t, V, I_e, I_i = backup_spl.get_tdata_raw_iv()

fig, ax = plt.subplots()
I_tot = (I_e + I_i)[::100]
ax.plot(I_tot)

for i in range(4):
    line_pos = i * len(I_tot) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)


In [94]:
backup_spl.tdata.pot[360:380, 1050:].mean()

-0.3740846272751438

In [95]:
fig, ax = plt.subplots()
mapper = ax.pcolormesh(backup_spl.tdata.pot)
fig.colorbar(mapper)
ax.pcolormesh(backup_spl.tdata.pot[360:380, 1050:], cmap='plasma')

In [99]:
backup_spl.plot_2d_variable(b_arrow_loc=(100, 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-99-e7aac73b46db>", line 1, in <module>
    backup_spl.plot_2d_variable(b_arrow_loc=(100, 100))
  File "/home/jleland/coding/projects/flopter/flopter/spice/splopter.py", line 657, in plot_2d_variable
    plt.imshow(objects, cmap='Greys', extent=[0, len(plasma_parameter[0]) / 2, 0, len(plasma_parameter) / 2])
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/pyplot.py", line 2683, in imshow
    None else {}), **kwargs)
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/__init__.py", line 1601, in inner
    return func(ax, *map(sanitize_sequence, args), **kwargs)
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/cbook/depre

OSError: [Errno 5] Input/output error

In [98]:
fig, ax = plt.subplots()

print(backup_spl.tdata.pot.shape)
ax.plot(backup_spl.tdata.pot[:,200])

(465, 1113)


In [75]:
def plot_2d_slice(splopter, var_2d='Pot', x_slice=200, ax=None, invert_fl=False):
    if ax is None:
        fig, ax = plt.subplots(2, figsize=[6,12])
    else: 
        fig = ax[0].figure
        
    var_2d_data = backup_spl.tdata.t_dict[var_2d]
    if invert_fl:
        var_2d_data = -var_2d_data

    ax[0].contourf(var_2d_data)
    ax[0].axvline(x=x_slice, **c.AX_LINE_DEFAULTS)
    ax[0].axis('scaled')
    ax[0].autoscale()

    ax[1].plot( np.arange(var_2d_data.shape[0]), var_2d_data[:, x_slice],)
#     ax[1].axis('scaled')
#     ax[1].autoscale()

    fig.tight_layout()
    return ax

def plot_slice(splopter, var_2d='Pot', x_slice=200, ax=None, invert_fl=False):
    if ax is None:
        fig, ax = plt.subplots(figsize=[6, 6])
    else: 
        fig = ax.figure
        
    var_2d_data = backup_spl.tdata.t_dict[var_2d]
    if invert_fl:
        var_2d_data = -var_2d_data

    ax.plot( np.arange(var_2d_data.shape[0]), var_2d_data[:, x_slice],)
#     ax[1].axis('scaled')
#     ax[1].autoscale()

    fig.tight_layout()
    return ax

def plot_mean_slice(splopter, var_2d='Pot', x_slice=slice(180,220), ax=None, invert_fl=False):
    if ax is None:
        fig, ax = plt.subplots(figsize=[6, 6])
    else: 
        fig = ax.figure
        
    var_2d_data = backup_spl.tdata.t_dict[var_2d]
    if invert_fl:
        var_2d_data = -var_2d_data

    ax.plot( np.arange(var_2d_data.shape[0]), np.mean(var_2d_data[:, x_slice], axis=1),)
#     ax[1].axis('scaled')
#     ax[1].autoscale()

    fig.tight_layout()
    return ax

In [84]:
plot_mean_slice(backup_spl, x_slice=(0,120), var_2d='rho01')

In [65]:
plot_2d_slice(backup_spl, x_slice=30, var_2d='Pot')

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f493da954a8>,
      dtype=object)

In [82]:
fig, ax = plt.subplots(2, 3, figsize=[14, 10], sharey='row')

plot_2d_slice(backup_spl, var_2d='rho', ax=ax[:, 0], x_slice=400)
plot_2d_slice(backup_spl, var_2d='rho01', ax=ax[:, 1])
plot_2d_slice(backup_spl, var_2d='rho02', ax=ax[:, 2], invert_fl=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f493af3eeb8>,
      dtype=object)

In [92]:
fig, ax = plt.subplots(3, figsize=[10, 10])


for i, x_slc in enumerate([20, 200, 300]):
    plot_slice(backup_spl, var_2d='rho01', x_slice=x_slc, ax=ax[i])
    plot_slice(backup_spl, var_2d='rho02', x_slice=x_slc, ax=ax[i], invert_fl=True)
    ax[i].set_xlim(90, 250)

In [83]:
x_slices = [0, 280]
samples = [60, 150]
n_axes = len(x_slices)

fig, ax = plt.subplots(n_axes + 1, figsize=[10, 10])
rho01 = backup_spl.tdata.t_dict['rho01']
rho02 = -backup_spl.tdata.t_dict['rho02']

for i, (x_slc, sample) in enumerate(zip(x_slices, samples)):    
    if x_slc - sample < 0:
        roll_offs = np.abs(x_slc-sample)
        rho01 = np.roll(backup_spl.tdata.t_dict['rho01'], roll_offs)
        rho02 = np.roll(-backup_spl.tdata.t_dict['rho02'], roll_offs)
        x_slc = x_slc + roll_offs
    x_slice = slice(x_slc-sample, x_slc+sample)
    ax[i].plot(np.arange(rho01.shape[0]), np.mean(rho01[:, x_slice], axis=1),)
    ax[i].plot(np.arange(rho02.shape[0]), np.mean(rho02[:, x_slice], axis=1),)
#     plot_mean_slice(backup_spl, var_2d='rho01', x_slice=slice(x_slc-sample, x_slc+sample), ax=ax[i])
#     plot_mean_slice(backup_spl, var_2d='rho02', x_slice=slice(x_slc-sample, x_slc+sample), ax=ax[i], invert_fl=True)
    ax[i].set_xlim(93, 220)
    
    ax[n_axes].plot(np.arange(rho01.shape[0]), np.mean(rho01[:, x_slice], axis=1) - np.mean(rho02[:, x_slice], axis=1),)
    ax[n_axes].set_xlim(92, 220)
    ax[n_axes].set_ylim(-1.2, 5)
#     ax[n_axes].axvline(x=x_slc, **c.AX_LINE_DEFAULTS)
#     ax[n_axes].axvspan(x_slc + sample, x_slc - sample, color='grey', alpha=0.5)
    
# ax[n_axes].pcolormesh(backup_spl.tdata.pot)

fig.tight_layout()


In [112]:
fig, ax = plt.subplots()
ax.pcolormesh(backup_spl.tdata.pot[:, slice(0, 0-60)])

In [116]:
L = backup_spl.parser.getfloat('rectangle2', 'yhigh') - backup_spl.parser.getfloat('rectangle2', 'ylow')
g = backup_spl.parser.getfloat('rectangle2', 'ylow') - backup_spl.parser.getfloat('rectangle0', 'yhigh')
sim_w = backup_spl.parser.getfloat('geom', 'Ly')
print(sim_w)

556.0


In [117]:
h = backup_spl.parser.getfloat('triangle0', 'zc') - backup_spl.parser.getfloat('rectangle1', 'zhigh')
L_back = backup_spl.parser.getfloat('rectangle1', 'yhigh') - backup_spl.parser.getfloat('rectangle1', 'ylow')
L_front = backup_spl.parser.getfloat('rectangle0', 'yhigh') - backup_spl.parser.getfloat('rectangle0', 'ylow')
# theta = np.abs(backup_spl.parser.getfloat('plasma', 'alpha_yz'))
theta = 2
L_shadow = np.ceil((h / np.tan(np.radians(theta))) - g)

In [118]:
print(h)
print(g)
print(L_front + L_back)
print(L_shadow)

35.0
60.0
136.0
943.0


In [119]:
print(np.ceil(L_shadow))
print(np.ceil(L_shadow) - L_back) # length to add to rear tile
print(sim_w + np.ceil(L_shadow) - L_back) # new sim width

943.0
875.0
1431.0


## Comparing 2 runs

In [27]:
compare_spl_path = spice_dir / all_run_dirs[scans[0]][0]
print(compare_spl_path)
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled/alpha_yz_-10.0
/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled_longtail_10/alpha_yz_-10.0


In [28]:
comp_spl = spl.Splopter(compare_spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                        store_dataframe_fl=True, check_voltage_error_fl=False)
comp_spl.parse_input()

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled/alpha_yz_-10.0) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [29]:
t_1, V_1, I_e_1, I_i_1 = backup_spl.get_tdata_raw_iv()

fig, ax = plt.subplots()
ax.plot((I_e + I_i)[::100])
ax.plot((I_e_1 + I_i_1)[::100])

In [31]:
backup_iv, _ = backup_spl.homogenise()
comp_iv, _ = comp_spl.homogenise()

In [55]:
fig, ax = plt.subplots()

backup_iv.plot(ax=ax)
comp_iv.plot(ax=ax)

<ErrorbarContainer object of 3 artists>

In [57]:
fig, ax = plt.subplots()

for splop in [comp_spl.iv_df, backup_spl.iv_df]:
    sploop = splop.to_xarray().swap_dims({'index':'voltage'}).sel(voltage=slice(-9.95,-3.4))
#     ax.plot(np.float_power(np.abs(sploop['voltage']), 0.75), sploop['current_i'])
    
    fitter = fts.IonCurrentSEFitter()
    fitter.fit(np.float_power(np.abs(sploop['voltage'].values), 0.75), sploop['current_i'].values).plot(ax=ax)
# ax.plot(np.float_power(np.abs(comp_spl.iv_df['voltage']), 0.75), comp_spl.iv_df['current_i'])
# ax.plot(np.float_power(np.abs(backup_spl.iv_df['voltage']), 0.75), backup_spl.iv_df['current_i'])

## Look at the Ion Distribution Function at Injection

In [202]:
spice_bin_path = pth.Path('/home/jleland/spice/spice2/bin/')

In [263]:
ion_dfs = list(spice_bin_path.glob('fu_iz*'))
ion_dfs.extend(list(spice_bin_path.glob('fu_ez*')))

In [264]:
ion_dfs.extend(list(spice_bin_path.glob('DF/fu_iz*')))
ion_dfs.sort()

In [265]:
ion_dfs

[PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU01.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU02.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU03.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU04.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU05.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU06.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU07.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU08.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU09.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU10.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU11.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU12.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU13.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU14.dat'),
 PosixPath('/home/jleland/spice/spice2/bin/DF/fu_iz_TAU15.dat'),
 PosixPath('/home/jleland

In [266]:
ion_df_try = np.loadtxt(ion_dfs[0])
print(ion_dfs[0])
ion_len = int(ion_df_try[0])
ion_x = ion_df_try[1:ion_len + 1]
ion_y = ion_df_try[ion_len + 1:]
print(ion_len, len(ion_x), len(ion_x))
print(ion_x)
print(ion_y)

/home/jleland/spice/spice2/bin/DF/fu_iz_TAU01.dat
121 121 121
[-2.0146038  -1.9978154  -1.9810271  -1.9642387  -1.9474503  -1.930662
 -1.9138736  -1.8970852  -1.8802969  -1.8635085  -1.8467202  -1.8299318
 -1.8131434  -1.7963551  -1.7795667  -1.7627783  -1.74599    -1.7292016
 -1.7124132  -1.6956249  -1.6788365  -1.6620481  -1.6452598  -1.6284714
 -1.611683   -1.5948947  -1.5781063  -1.5613179  -1.5445296  -1.5277412
 -1.5109529  -1.4941645  -1.4773761  -1.4605878  -1.4437994  -1.427011
 -1.4102227  -1.3934343  -1.3766459  -1.3598576  -1.3430692  -1.3262808
 -1.3094925  -1.2927041  -1.2759157  -1.2591274  -1.242339   -1.2255506
 -1.2087623  -1.1919739  -1.1751856  -1.1583972  -1.1416088  -1.1248205
 -1.1080321  -1.0912437  -1.0744554  -1.057667   -1.0408786  -1.0240903
 -1.0073019  -0.99051354 -0.97372517 -0.95693681 -0.94014844 -0.92336008
 -0.90657171 -0.88978335 -0.87299498 -0.85620662 -0.83941825 -0.82262989
 -0.80584152 -0.78905316 -0.77226479 -0.75547643 -0.73868806 -0.7218997
 -

In [267]:
fig, ax = plt.subplots()

ax.plot(ion_x, ion_y)

In [268]:
(int(ion_dfs[9].name.split('_')[2][3:5]) / 10) % 1

0.0

In [278]:
def plot_ion_df(ion_df_arr, ax=None, **kwargs):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    
    ion_len = int(ion_df_arr[0])
    ion_x = ion_df_arr[1:ion_len + 1]
    ion_y = ion_df_arr[ion_len + 1:]
    
    ax.plot(-ion_x, ion_y, **kwargs)
    
    return ax


In [342]:
fig, ax = plt.subplots(2, sharey=True)

for ion_df_path in ion_dfs:
    plot_ax = ax[0]
    
    if 'DF' in str(ion_df_path):
        tau = int(ion_df_path.name.split('_')[2][3:5]) / 10
        if tau % 0.5 != 0.0:
            continue
    else:
        tau = ion_df_path.name
        if 'ez' in str(ion_df_path):
            plot_ax = ax[1]
    
    ion_df_arr = np.loadtxt(ion_df_path)
    plot_ion_df(ion_df_arr, ax=plot_ax, label=tau)
    
ax[0].legend()
ax[0].axvline(x=1.0, **c.AX_LINE_DEFAULTS)
ax[1].legend()
ax[1].axvline(x=1.0, **c.AX_LINE_DEFAULTS)

In [285]:

ax = plot_ion_df(np.loadtxt(ion_dfs[9]))
ax.axvline(x=1.0, **c.AX_LINE_DEFAULTS)
ax.set_ylabel(r'$f(v) \; [normalised]$')
ax.set_xlabel(r'$v \; [c_s]$')

Text(0.5, 0, '$v \\; [c_s]$')

In [294]:
ion_df_arr = np.loadtxt(ion_dfs[9])

ion_len = int(ion_df_arr[0])
ion_x = -ion_df_arr[1:ion_len + 1]
ion_y = ion_df_arr[ion_len + 1:]


In [287]:
import scipy.integrate as integrate
import scipy.interpolate as intp

In [293]:
ion_func = intp.interp1d(ion_x, ion_y)
integrate.quad(ion_func, 1, max(-ion_x))

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  


(0.29608074793932376, 8.969926142484214e-06)

In [333]:
fitter = fts.MaxwellianVelFitter()
fit_data = fitter.fit(ion_x, ion_y, initial_vals=[.0001, 0.0])

/home/jleland/coding/projects/flopter/flopter/core/fitters.py:531: RuntimeWarning: divide by zero encountered in double_scalars
  a = self.mass / (2 * self.temp_conversion * T_e)
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:532: RuntimeWarning: invalid value encountered in multiply
  return np.power(a / np.pi, 1.5) * 4 * np.pi * np.power(v, 2) * np.exp(-a * np.power(v, 2))


In [334]:
fit_data.plot()

In [335]:
fig, ax = plt.subplots()
ax.plot(ion_x, fitter.fit_function(ion_x, *[.0000000011, 1.0]))

In [341]:
total_area = integrate.simps(ion_y, dx=0.0363859)
bohm_area = integrate.quad(ion_func, 1, max(ion_x))
bohm_area[0] / total_area

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  


0.7766836656399815